# Character-based LSTM: Training

In [2]:
from LSTMforSonnet import LSTM_char
from keras.models import Sequential
from keras.layers import Dense

numEpoch = 400
stepList = [1, 2, 4, 8]
patienceList = [5]
for step in stepList:
    for patience in patienceList:
        test = LSTM_char(step=step)
        test.SonnetLoader('shakespeare')
        test.getTrainSeq()
        test.getMapping()
        test.Train(numEpoch = numEpoch, patience = patience, fileName = "model_step%d_patience%d_maxEpoch%d" % (step, patience, numEpoch))

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 200)               217600    
_________________________________________________________________
dense_1 (Dense)              (None, 71)                14271     
Total params: 231,871
Trainable params: 231,871
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/400


KeyboardInterrupt: 

 # Character-based LSTM: Prediction


In [3]:
# make predictions after loading trained model from saved files
from LSTMforSonnet import LSTM_char
from keras.models import Sequential
from keras.layers import Dense

test = LSTM_char()
step = 1
patience = 15
maxEpoch = "_maxEpoch400"
modelName = "model_step%d_patience%d%s_char.h5" % (step, patience, maxEpoch)
mappingName = "model_step%d_patience%d%s_char.pkl" % (step, patience, maxEpoch)
print(modelName)
print(mappingName)
#test.LoadModel(modelName = modelName, mappingName = mappingName)
test.LoadModel()
tempList = [1.5, 1, 0.75, 0.25]
predicted = [test.Predict("shall i compare thee to a summer's day?\n", outputText_len=800, temperature = x) for x in tempList]
for i, x in enumerate(predicted):
    print('with temperature ', tempList[i], '\n', x)


model_step1_patience15_maxEpoch400_char.h5
model_step1_patience15_maxEpoch400_char.pkl
with temperature  1.5 
 shall i compare thee to a summer's day?
The merit his spilf belies with flowns Where's dost see,
Thau art thou lood, and teve, the ripe with like,
  To to tillen in toured dooks hir the hall,
And shores the hole all bean hese'st?
I steet my beauty, whore is not lives be,
In tince, some in touring braily is mine,
The repelf of surf io the eresteting ste,
For now to fillfrow hatcoring thate,
When I ait ins thee I an eymine ere I see,
Seat me that some vorterece,
My loviese, and tide my hall beauty,
Those will twile see in my have reser sheling;
Rothis can is reberity in loue y see thee,
  I s most glow, thos true with wrim thech oke,
That mist by vere with my there I wase grower prome.
  Thoo thyid, 's not thee,  houll to the palledss,
Ore thin thy betull hasu are now love,
  That acceppet in thing thy forter best de.
Het new repo
with temperature  1 
 shall i compare thee to a 

 # Word-based LSTM: Training

In [ ]:
from LSTMforSonnet import LSTM_char, LSTM_word
from keras.models import Sequential
from keras.layers import Dense

useWordEmbedding = True
embedSize = [100, 200]
numUnit = [200, 400]
for dim in embedSize:
    for un in numUnit:
        test2 = LSTM_word(LSTM_numUnits = un)
        #test2.SonnetLoader('shakespeare')
        #test2.SonnetLoader('Spenser')
        test2.SonnetLoader(['shakespeare', 'spenser'])
        test2.getTrainSeq()
        test2.getMapping()
        test2.Train(useWordEmbedding = useWordEmbedding, embeddingSize = dim, numEpoch = 200, fileName = "model_Both_unit%d" % un)

 # Word-based LSTM: Prediction

In [4]:
from LSTMforSonnet import LSTM_char, LSTM_word
from keras.models import Sequential
from keras.layers import Dense

## model with minimum perplexity
useWordEmbedding = True
checkPentameter = False
embedSize = [200]
numUnit = [400]
data = 'both'

if data == 'shakespeare':
    fileName_data = ''
elif data == 'spenser':
    fileName_data = 'Spenser_'
elif data == 'both':
    fileName_data = 'Both_'

for i, dim in enumerate(embedSize):
    for j, un in enumerate(numUnit):
        if useWordEmbedding:
            modelName = "model_%sunit%d_withWordEmbedding %d.h5" % (fileName_data, un, dim)
            mappingName = "model_%sunit%d_mapping_withWordEmbedding %d.pk1" % (fileName_data, un, dim)
        test2 = LSTM_word()
        test2.LoadModel(modelName = modelName, mappingName = mappingName)
        if data == 'shakespeare':
            test2.SonnetLoader('shakespeare')
        elif data == 'spenser':
            test2.SonnetLoader('spenser')
        elif data == 'both':
            test2.SonnetLoader(['shakespeare', 'spenser'])
        
        #tempList = [1.5, 1, 0.75, 0.25]
        tempList = [1, 0.75, 0.25]
        predicted = [test2.Predict("shall i compare thee to a summer's day?\n", outputText_len=150, temperature = x, checkPentameter = checkPentameter, useWordEmbedding = useWordEmbedding) for x in tempList]
        for i, x in enumerate(predicted):
            print('temperature: ', tempList[i], ', embedding size: ', dim, '\n', x, '\n')


C:\Users\HyeongChan Jo\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 40, 200)           874600    
_________________________________________________________________
lstm_8 (LSTM)                (None, 400)               961600    
_________________________________________________________________
lambda_1 (Lambda)            (None, 400)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 4373)              1753573   
Total params: 3,589,773
Trainable params: 3,589,773
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 40, 200)           874600    
______________________

## Compute perplexity and accuracy of trained models

In [1]:
from LSTMforSonnet import LSTM_char, LSTM_word
from keras.models import Sequential
from keras.layers import Dense
import Dictionary

embedSize = [5, 10, 25]
for dim in embedSize:
#     modelName = "model_Spenser_withWordEmbedding %d.h5" % dim
#     mappingName = "model_Spenser_mapping_withWordEmbedding %d.pk1" % dim
    modelName = "model_unit400_withWordEmbedding %d.h5" % dim
    mappingName = "model_unit400_mapping_withWordEmbedding %d.pk1" % dim
    test2 = LSTM_word()
    dic = Dictionary.syl_predef()
    #test2.SonnetLoader('Spenser_v2')
    test2.SonnetLoader('shakespeare')
    test2.getTrainSeq()
    test2.getMapping()
    test2.LoadModel(modelName = modelName, mappingName = mappingName)
    [perplexity, accuracy] = test2.perplexity_train(useWordEmbedding=True)
    print(perplexity)
    print(accuracy)

Using TensorFlow backend.


test
test2
3205


C:\Users\HyeongChan Jo\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 40, 5)             16030     
_________________________________________________________________
lstm_14 (LSTM)               (None, 400)               649600    
_________________________________________________________________
lambda_1 (Lambda)            (None, 400)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 3206)              1285606   
Total params: 1,951,236
Trainable params: 1,951,236
Non-trainable params: 0
_________________________________________________________________
13577/13577 [==============================] - 17s 1ms/step
[0.006680879098548909, 1.0]
perplexity:  1.006703245953655
accuracy:  1.0
1.006703245953655
1.0
test
test2
3205


C:\Users\HyeongChan Jo\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 40, 10)            32060     
_________________________________________________________________
lstm_15 (LSTM)               (None, 400)               657600    
_________________________________________________________________
lambda_2 (Lambda)            (None, 400)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 3206)              1285606   
Total params: 1,975,266
Trainable params: 1,975,266
Non-trainable params: 0
_________________________________________________________________
13577/13577 [==============================] - 17s 1ms/step
[4.1456098875768, 0.19901303946971893]
perplexity:  63.156128296312325
accuracy:  0.19901303946971893
63.156128296312325
0.19901303946971893
test
test2
3205


C:\Users\HyeongChan Jo\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 40, 25)            80150     
_________________________________________________________________
lstm_16 (LSTM)               (None, 400)               681600    
_________________________________________________________________
lambda_3 (Lambda)            (None, 400)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 3206)              1285606   
Total params: 2,047,356
Trainable params: 2,047,356
Non-trainable params: 0
_________________________________________________________________
 9568/13577 [====================>.........] - ETA: 4s

KeyboardInterrupt: 